In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from PIL import Image

In [2]:
opt = {
    'image_size': 32,
    'is_grayscale': True,
    'val_split': 0.75
}

In [3]:
def load_image(path):
    im = Image.open(path).convert('L' if opt['is_grayscale'] else 'RGB')
    im = im.resize((opt['image_size'],opt['image_size']))
    im = np.array(im)
    return im

def display_images(imgs,row=1,col=2,w=32,h=32):
    fig=plt.figure(figsize=(8, 8))
    for i in range(1, col*row +1):
        img = imgs[i-1]
        fig.add_subplot(row, col, i)
        
        if opt['is_grayscale']:
            plt.imshow(img , cmap='gray') 
        else:
            plt.imshow(img)
        
        # plt.title("Class:{}".format(classes[i-1]))
        plt.axis('off')
    plt.show()

def load_data(dir_path):
    image_list = []
    for filename in sorted(os.listdir(dir_path)):
        if filename.endswith(".png"):
            im = load_image(os.path.join(dir_path,filename))
            image_list.append(im)
        else:
            continue
    image_list = np.array(image_list)
    print("Dataset shape:",image_list.shape)

    return image_list

### 4.1 KIMIA Dataset ###

In [4]:
path = './OneDrive_2023-09-24/KIMIA dataset/'
X = load_data(path)

Dataset shape: (99, 32, 32)


In [5]:
import cv2
N,H,W = X.shape[0:3]
C = 1 if opt['is_grayscale'] else X.shape[3]

np.shape(X[0])

(32, 32)

In [7]:
from sklearn.decomposition import PCA
import cv2

# Load the template shape and extract orientation information
directory = './OneDrive_2023-09-24/KIMIA dataset'
template = 'cat1.png'

template_image = load_image(os.path.join(directory, template))

# black_pixels_template = np.where(template_image == 0)
black_pixels_template = []
for i in range(template_image.shape[0]):
    for j in range(template_image.shape[1]):
        if (template_image[i][j] == 0):
            black_pixels_template.append([i, j])

# black_pixels_template = np.transpose(black_pixels_template)

pca = PCA(n_components=2)
x = []
pca.fit_transform(black_pixels_template)
eigenvectors_X = pca.components_[1]

image_x = Image.open(os.path.join(directory, template))
output_folder = "./output"  # Replace with the path to your output folder
output_file = f"{output_folder}/image{0}.jpg"
image_x.save(output_file)

k = 1
for image in sorted(os.listdir(directory)):
    if (image == template):
        continue

    image_rotate = load_image(os.path.join(directory, image))    
    # black_pixels = np.where(image_rotate == 0)
    # black_pixels = np.transpose(black_pixels)

    black_pixels = []
    for i in range(image_rotate.shape[0]):
        for j in range(image_rotate.shape[1]):
            if (image_rotate[i][j] == 0):
                black_pixels.append([i, j])

    pca1 = PCA(n_components=2)
    pca1.fit_transform(black_pixels)
    eigenvectors_Y = pca1.components_[1]
    
    angle = np.arccos(np.dot(eigenvectors_X, np.transpose(eigenvectors_Y)))
    angle = np.degrees(angle)
    height, width = template_image.shape[:2]

    x = np.cross(eigenvectors_X, eigenvectors_Y)

    rotated_image = Image.open(os.path.join(directory, image))
    if (x >= 0):
        rotated_image = rotated_image.rotate(angle)
    else:
        rotated_image = rotated_image.rotate(-angle)
    
    output_folder = "./output"  # Replace with the path to your output folder
    output_file = f"{output_folder}/image{k}.jpg"
    rotated_image.save(output_file)
    k += 1

### Flowchart for Aligning Shapes with a Template (KIMIA-99 Dataset) ###

1. Load Template Image and KIMIA-99 Dataset:
   Begin by loading the template shape image and the KIMIA-99 dataset, which contains 99 different shapes.

2. Initialize Variables and Set Output Folder:
   Initialize essential variables, counters, and specify an output folder to store the aligned images.

3. Iterate Through Each Shape in the Dataset:
   For every shape in the KIMIA-99 dataset:
     - Check if the shape matches the template; if it does, move to the next shape.
     - Calculate the coordinates of black pixels in the template image using PCA (Principal Component Analysis).
     - Determine the orientation (angle) of the template shape.

4. Process Each Shape in the Dataset:
   For each shape in the dataset (excluding the template):
     - Compute the coordinates of black pixels in the current shape image using PCA.
     - Determine the orientation (angle) of the current shape.
     - Calculate the angle difference between the template and current shape orientations.
     - Decide on the rotation direction (clockwise or counterclockwise) based on the angle difference.
     - Rotate the current shape image to align it with the template shape, following the determined rotation direction.

5. Save Aligned Images:
   Store the aligned images in the specified output folder with unique filenames.

6. Repeat for All Shapes:
   Reiterate steps 3 to 5 for all shapes present in the dataset.

7. Algorithm Completion:
   The algorithm concludes after processing all shapes in the dataset.

8. Output:
   The output folder now contains aligned images of the dataset shapes, aligned with respect to the template shape.
